# Modules requirements

## Install modules

In [ ]:
"""
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip3 install 'memory_profiler'
!pip3 install 'line_profiler'
"""

"\n!pip3 install 'torch==1.3.1'\n!pip3 install 'torchvision==0.5.0'\n!pip3 install 'Pillow-SIMD'\n!pip3 install 'tqdm'\n!pip3 install 'memory_profiler'\n!pip3 install 'line_profiler'\n"

## Import modules and define auxiliary funcs

In [ ]:
import os
import logging
import copy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from matplotlib.pyplot import imshow
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

import tensorflow as tf
import torch.nn.functional as F
import datetime
#import memory_profiler
import functools, time
#import line_profiler
#from memory_profiler import profile
import os

#%load_ext memory_profiler

# helper functions for pytorch->tensorboard integration
# helper function to show an image
# (used in the `plot_classes_preds` function below)
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

def images_to_probs(net, images):
  #TODO
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output, discriminator_outputs = net(images, depth_images, forward_to_M=True, forward_to_P=True)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.cpu().numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[int(preds[idx])],
            probs[idx] * 100.0,
            classes[int(labels[idx])] ),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

def make_weights_for_balanced_classes(images, nclasses):
  """Get weight for class balancing"""
  count = [0] * nclasses                                                      
  for item in images:                                                         
    count[item[1]] += 1                                                     
  weight_per_class = [0.] * nclasses                                      
  N = float(sum(count))                                                   
  for i in range(nclasses):                                                   
    weight_per_class[i] = N/float(count[i])                                 
  weight = [0] * len(images)                                              
  for idx, val in enumerate(images):                                          
    weight[idx] = weight_per_class[val[1]]                                  
  return weight


!rm -rf ColabHelper
# Import custom module for improved Colab
if not os.path.isdir('./ColabHelper'):
  !git clone https://github.com/Emmunaf/ColabHelper.git
from ColabHelper.colabhelper import ColabHelper

!rm -rf RGBD_DA
# Import custom dataloader 
if not os.path.isdir('./RGBD_DA'):
  !git clone https://github.com/Emmunaf/RGBD_DA.git
from RGBD_DA.datasetloader import *

Cloning into 'ColabHelper'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 111 (delta 35), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (111/111), 30.77 KiB | 177.00 KiB/s, done.
Resolving deltas: 100% (35/35), done.
Cloning into 'RGBD_DA'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 129 (delta 42), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (129/129), 31.93 KiB | 198.00 KiB/s, done.
Resolving deltas: 100% (42/42), done.


In [ ]:
#
#@timeit
def download_dataset(gdrive=False):
  """ Download the SynROD and ROD datasets if the folder DS doesnt exist.
  
      Download from GDRIVE/MLDS/ folder if gdrive=True
      from a mirror server otherwise.
      Note: Gdrive often limit the bandwidth for big files.
  """

  if os.path.isdir('./DS'):
     return ('./DS/synROD/', './DS/ROD/')

  if gdrive:
    drive.mount('/content/drive/')
    !cp '/content/drive/My Drive/MLDS/ROD.zip' ROD.zip #%time
    !cp '/content/drive/My Drive/MLDS/synROD.zip' synROD.zip
  else:
    !wget https://www.loconsegno.io/MLDS/ROD.zip
    !wget https://www.loconsegno.io/MLDS/synROD.zip
    
  # Create dataset folder
  !mkdir "DS"

  # Unzip the downloaded datasets
  !unzip ROD.zip -d ./DS/
  !rm ROD.zip
  !unzip synROD.zip -d ./DS/
  !rm synROD.zip

  # Copy or download the ROD split file
  if gdrive:
    !cp '/content/drive/My Drive/MLDS/rod-split_sync.txt' ./DS/ROD/rod-split_sync.txt
  else:
    !wget https://www.loconsegno.io/MLDS/rod-split_sync.txt -O ./DS/ROD/rod-split_sync.txt
  return ('./DS/synROD/', './DS/ROD/')


## Load extensions

In [ ]:
# Initialize colabhelper object 
ch = ColabHelper(backup_folder="/content/drive/My Drive/MLDS/FinalProject_runsBackup/")
# Set notification parameters for authentication pourpose
# ** NEVER STORE your usertoken on public domain. Use Form field of colab eventually **
pushover_params = {"user_token":"u8u62zneyk2b6t264kjh7d9nvkb9sj", "app_token":"a4zdx6q46hn9uca9rrycjt7m5xkjoo"}
ch.set_notification_params(service="pushover", params=pushover_params)
ch.get_gpu_info()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[WARN] Your backup folder /content/drive/My Drive/MLDS/FinalProject_runsBackup/ already exists. Some files could be overwritten
[INFO] A tensorboard backup folder was detected ( /content/drive/My Drive/MLDS/FinalProject_runsBackup/runs )
You can use the tensorboard_restore() method to load it.
Fri Jun 26 10:26:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======

In [ ]:
# Load the TensorBoard notebook extension and memory profiler module for debug
%load_ext tensorboard
#%load_ext memory_profiler
#%load_ext line_profiler
%tensorboard --logdir runs  # Lunch tensorboard with logfolder param="runs"
# TO the reader: sometimes, on colab, tensorboard go out of memory or became stucked. 
# This behaviour was not detected in the local environment. By the way is easy to disable tensorflow logging
# just put writer=None instead of writer=SummaryWriter(...)

Reusing TensorBoard on port 6006 (pid 1345), started 1:15:24 ago. (Use '!kill 1345' to kill it.)

<IPython.core.display.Javascript object>

# Net hyperparameters


# Hyperparameters used in the paper

learning rate: 0.0003

learning rate decay: fixed (no decay, no stepdown)

batch size: 64

main task weight (classification on labeled images): 1.0

rotation loss weight: 1.0

entropy loss weight: 0.1

sgd momentum: 0.9

weight decay: 0.05

dropout: 0.5

epochs: 40

In [ ]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

# Dataset and preprocessing params
N_SAMPLES_S = 41700
N_SAMPLES_T = 18000
MIN_SIZE_IN_IMG = 55
TRAIN_VAL_RATIO = 0.94
NUM_CLASSES = 47
NUM_ROTATION = 4

LAMBDA = 0.1
BATCH_SIZE = 32     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results
LR = 3e-4           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-2  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10
VAL_FREQUENCY = 150
DROP_OUT = 0.5
NESTEROV = False

## Define transformations

In [ ]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.RandomCrop(224),  # Crops a  square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(224),
                                      #transforms.RandomCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])                                    
])


## Import dataset

In [ ]:
#https://drive.google.com/drive/folders/1_NAcoR0UGH1eLsiWMOx_Py8yeAocknA2
DATASET_SOURCE_PATH, DATASET_TARGET_PATH = download_dataset(gdrive=False)

## Prepare source and target dataset and dataloaders

In [ ]:
# Load source dataset (synthetic rod)
blacklist = ['lime', 'onion', 'peach', 'tomato']  # These labels can be found only on synrod => remove
source_dataset = SynROD(DATASET_SOURCE_PATH,  transform=train_transform, blacklisted_classes=blacklist, min_width=MIN_SIZE_IN_IMG, min_height=MIN_SIZE_IN_IMG, n_samples=N_SAMPLES_S)

# Define source dataset dimension and subsets proportion
source_indexes, _ = source_dataset.reduce_data(TRAIN_VAL_RATIO)  # Eventually reduce dataset to fit in RAM. (0.66628->32476 as full target without n_samples)

source_train_indexes = source_indexes
'''
source_train_indexes, source_testval_indexes = source_dataset.split_data(0.5)# split the indices for train and val split
source_val_indexes = [idx for idx in source_testval_indexes if idx % 3]
source_test_indexes = [idx for idx in source_testval_indexes if not idx % 3]
'''

# Define obj labels and training dataloader
classes = source_dataset.get_classes()
source_train_datasubset = Subset(source_dataset, indices=source_train_indexes )
source_train_dataloader = DataLoader(source_train_datasubset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

# Define validation and testing dataloader
#source_val_datasubset = Subset(source_dataset, indices=source_val_indexes )
#source_val_dataloader = DataLoader(source_val_datasubset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
#source_test_datasubset = Subset(source_dataset, indices=source_test_indexes )
#source_test_dataloader = DataLoader(source_test_datasubset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

print('[INFO] Source Train Dataset: {}'.format(len(source_train_datasubset)))
#print('[INFO] Source Validation Dataset: {}'.format(len(source_val_datasubset)))
#print('[INFO] Source Test Dataset: {}'.format(len(source_test_datasubset)))

[INFO] A total of  3878  samples were skipped because of of min_width parameter
[INFO] A total of  3878  samples were skipped because of of min_height parameter
[INFO] A total of  135 samples were skipped because their depth map it's missing
[INFO] Source Train Dataset: 35553


In [ ]:
# Load target dataset (real rod)
blacklist = []
target_dataset = ROD(DATASET_TARGET_PATH, split='rod-split_sync',  transform=train_transform, blacklisted_classes=blacklist, min_width=MIN_SIZE_IN_IMG, min_height=MIN_SIZE_IN_IMG, n_samples=N_SAMPLES_T)

# Define target dataset dimension and subsets proportion
#target_indexes = [idx for idx in range(len(target_dataset))]
target_train_indexes, target_testval_indexes = target_dataset.split_data(1.0-TRAIN_VAL_RATIO)# split the indices for train and val split
#target_val_indexes = [idx for idx in target_testval_indexes if idx % 3]
#target_test_indexes = [idx for idx in target_testval_indexes if not idx % 3]

# Define target training dataloader
#classes = target_dataset.get_classes()  # already defined, same classes in both target and source ofc
target_train_datasubset = Subset(target_dataset, indices=target_train_indexes )
target_train_dataloader = DataLoader(target_train_datasubset, batch_size=BATCH_SIZE, num_workers=4, drop_last=True)

# Define target validation and testing subsets and the related dataloader
target_val_datasubset = Subset(target_dataset, indices=target_testval_indexes )
target_val_dataloader = DataLoader(target_val_datasubset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
#target_test_datasubset = Subset(target_dataset, indices=target_test_indexes )
#target_test_dataloader = DataLoader(target_test_datasubset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
target_test_datasubset = target_val_datasubset
target_test_dataloader = DataLoader(target_val_datasubset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

# Print dataset sizes
print('[INFO] target Train Dataset: {}'.format(len(target_train_datasubset)))
print('[INFO] target Validation Dataset: {}'.format(len(target_val_datasubset)))
#print('[INFO] target Test Dataset: {}'.format(len(target_test_datasubset)))

[INFO] A total of  687  samples were skipped because of of min_width parameter
[INFO] A total of  687  samples were skipped because of of min_height parameter
[INFO] A total of  0 samples were skipped because of a missing partner domain
[INFO] target Train Dataset: 16274
[INFO] target Validation Dataset: 1039


In [ ]:
#ch.tensorboard_backup()
#ch.tensorboard_restore()
pushover_params = {"user_token":"u8u62zneyk2b6t264kjh7d9nvkb9sj", "app_token":"a4zdx6q46hn9uca9rrycjt7m5xkjoo"}
ch.set_notification_params(service="pushover", params=pushover_params)
ch.notify(extra="RGBD-A paper implementation now works!")
#ch.beep()

## Model


## Model definition

Implementation of https://arxiv.org/abs/2004.10016


### Features extractors
Two identical CNNs, E c and E d , are used to process the RGB and depth image, respectively. The outputs of these two networks are then concatenated along the channel dimension to compose the final RGB-D feature. For our experiments, we define E c and E d as the ResNet- 18 [35] architecture without the final fully connected and global average pooling layers.

### Main head
The network M solves a C-way classification problem, where C indicates the number of object classes we want to predict. It is defined as [gap, fc(1000), fc(C)], where gap indicates a global average pooling operation, fc(n) indicates a fully connected layer with n neurons. fc(1000) uses batch normalization and ReLU activation function, while fc(C) uses softmax activation function.
### Pretexthead
The network P solves the 4-way classifi- cation problem of predicting the rotation between the RGB and depth image. It is defined as [conv(1 × 1,100), conv(3 × 3, 100), fc(100), fc(4)], where conv(k × k, n) indicates a 2D convolutional layer with kernel size k × k and n neurons. All convolutional and fully connected layers use batch normalization and ReLU activation function, except for fc(4) that uses softmax activation function. It is worth mentioning that, differently from M, we use convolutional layers in P to better preserve the spatial information.

In [ ]:
import torch
import torch.nn as nn
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch.nn import Parameter
from torch.autograd import Function

import torch.nn.functional as F

# nn.Reshape for sequential class implementation (https://github.com/pytorch/vision/issues/720)
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,  # extra comma for tuple support

    def forward(self, x):
        return x.view(*self.shape)

# Debug layer
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        print(x.shape)
        return x

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth'
}

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.ec_conv1 and self.ec_downsample layers downsample the input when stride != 1
        self.ec_conv1 = conv3x3(inplanes, planes, stride)
        self.ec_bn1 = norm_layer(planes)
        self.ec_relu = nn.ReLU(inplace=True)
        self.ec_conv2 = conv3x3(planes, planes)
        self.ec_bn2 = norm_layer(planes)
        self.ec_downsample = downsample
        self.ec_stride = stride

    def forward(self, x):
        identity = x

        out = self.ec_conv1(x)
        out = self.ec_bn1(out)
        out = self.ec_relu(out)

        out = self.ec_conv2(out)
        out = self.ec_bn2(out)

        if self.ec_downsample is not None:
            identity = self.ec_downsample(x)

        out += identity
        out = self.ec_relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.ec_conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.ec_conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.ec_conv2 and self.ec_downsample layers downsample the input when stride != 1
        self.ec_conv1 = conv1x1(inplanes, width)
        self.ec_bn1 = norm_layer(width)
        self.ec_conv2 = conv3x3(width, width, stride, groups, dilation)
        self.ec_bn2 = norm_layer(width)
        self.ec_conv3 = conv1x1(width, planes * self.expansion)
        self.ec_bn3 = norm_layer(planes * self.expansion)
        self.ec_relu = nn.ReLU(inplace=True)
        self.ec_downsample = downsample
        self.ec_stride = stride

    def forward(self, x):
        identity = x

        out = self.ec_conv1(x)
        out = self.ec_bn1(out)
        out = self.ec_relu(out)

        out = self.ec_conv2(out)
        out = self.ec_bn2(out)
        out = self.ec_relu(out)

        out = self.ec_conv3(out)
        out = self.ec_bn3(out)

        if self.ec_downsample is not None:
            identity = self.ec_downsample(x)

        out += identity
        out = self.ec_relu(out)

        return out


class RGBD_DA(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(RGBD_DA, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group

        # Define Ec layers
        self.ec_conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.ec_bn1 = norm_layer(self.inplanes)
        self.ec_relu = nn.ReLU(inplace=True)
        self.ec_maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.ec_layer1 = self._make_layer(block, 64, layers[0])
        self.ec_layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.ec_layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.ec_layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        #we define E_c and E_d as the ResNet- 18 architecture without the final fully connected and global average pooling layers.
        #self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        #self.fc = nn.Linear(512 * block.expansion, num_classes)
        # ** End of resnet **

        # Define Ed layers
        self.ed_conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.ed_bn1 = norm_layer(self.inplanes)
        self.ed_relu = nn.ReLU(inplace=True)
        self.ed_maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.ed_layer1 = self._make_layer(block, 64, layers[0])
        self.ed_layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.ed_layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.ed_layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        #we define E_c and E_d as the ResNet- 18 architecture without the final fully connected and global average pooling layers.
        #self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        #x = x.reshape(x.size(0), -1)
        #self.fc = nn.Linear(512 * block.expansion, num_classes)
        # ** End of resnet **
        
        #Ed/EC shape:  torch.Size([32, 512, 7, 7]).
        # Input (Combined) Shape:  torch.Size([32, 1024, 7, 7])
        self.mainhead = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=(1,1)), # [32768 x 1];         
            View((BATCH_SIZE, -1)), #should be the same as reshape(x.size(0), -1)
            nn.Linear(1024 * block.expansion, 1000), # fc(1000) uses batch normalization and ReLU activation function, {#size mismatch, m1: [32768 x 1], m2: [1024 x 1000] }
            nn.BatchNorm1d(num_features=1000),
            nn.ReLU(),
            nn.Linear(1000, NUM_CLASSES)
            #nn.Softmax(dim=1) #  fc(C) uses softmax activation function (included in crossentropy-based)
        )

        #[conv(1 × 1,100), conv(3 × 3, 100), fc(100), fc(4)]
        self.pretexthead = nn.Sequential(
            conv1x1(1024, 100),
            nn.BatchNorm2d(num_features=100),
            nn.ReLU(),
            conv3x3(100, 100),
            nn.BatchNorm2d(num_features=100),
            nn.ReLU(),
            View((BATCH_SIZE, -1)),
            nn.Linear(100*7*7, 100), # size mismatch, m1: [32 x 4900], m2: [100 x 100] 
            nn.BatchNorm1d(num_features=100),
            nn.ReLU(),
            #PrintLayer(),
            nn.Linear(100, NUM_ROTATION)
            #nn.Softmax(dim=1) #  fc(4) uses softmax activation function (included in crossentropy-based)
        )

        

        # *** ARES The following is used by resnet. For now let's try to apply to all layers not just Ec and Ed
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
        # *** /ARES 

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl_ec(self, x):
        # See note [TorchScript super()]
        x = self.ec_conv1(x)
        x = self.ec_bn1(x)
        x = self.ec_relu(x)
        x = self.ec_maxpool(x)

        x = self.ec_layer1(x)
        x = self.ec_layer2(x)
        x = self.ec_layer3(x)
        x = self.ec_layer4(x)

        #x = self.avgpool(x)
        #x = torch.flatten(x, 1)
        #x = self.fc(x)

        return x

    def _forward_impl_ed(self, x):
        # See note [TorchScript super()]
        x = self.ed_conv1(x)
        x = self.ed_bn1(x)
        x = self.ed_relu(x)
        x = self.ed_maxpool(x)

        x = self.ed_layer1(x)
        x = self.ed_layer2(x)
        x = self.ed_layer3(x)
        x = self.ed_layer4(x)

        #x = self.avgpool(x)
        #x = torch.flatten(x, 1)
        #x = self.fc(x)

        return x 

    def forward(self, x1, x2, forward_to_M, forward_to_P):
        """ 
        Forward method. X1 is the pure RGB tensor. X2 is the depth one.
        The other 2 parameters acts as a switch to forward to some or all heads (M and P)

        Note: if is_from_source = False, the output of EC and Ed combined is not forwarded to mainhead.
            The paper states that: M is trained for object recognition using (only) the labeled source data.
        """
        
        ec = self._forward_impl_ec(x1)
        ed = self._forward_impl_ed(x2)
        # print("Ec shape: ", ec.shape)
        # print("Ed shape: ", ed.shape)
        # From RGB and D -> RGB-D using stack as of now
        #combined = torch.stack(ec, ed, dim=0)
        combined = torch.cat((ec,ed), dim=1)
        # print("Combined Shape: ", combined.shape)
        if forward_to_M:
          class_outputs = self.mainhead(combined)
        else:
          class_outputs = None

        if forward_to_P:
          discriminator_outputs = self.pretexthead(combined)
        else:
          discriminator_outputs = None
        
        return class_outputs, discriminator_outputs
        #self.mainhead, self.pretexthead

    @staticmethod
    def _rename_layer(param_name):
      if "conv" in param_name or "bn" in param_name or "layer" in param_name or "downsample" in param_name :
        new_ec_pn = param_name.replace("conv", "ec_conv").replace("bn", "ec_bn").replace("layer", "ec_layer").replace("downsample", "ec_downsample")
        if len(param_name.split(".")) <= 1:
          new_ed_pn = param_name.replace("conv", "ed_conv").replace("bn", "ed_bn").replace("layer", "ed_layer").replace("downsample", "ed_downsample")
        else: 
          new_ed_pn = param_name.replace("conv", "ec_conv").replace("bn", "ec_bn").replace("layer", "ed_layer").replace("downsample", "ec_downsample")
        return new_ec_pn, new_ed_pn

      else:
        return param_name

    def custom_load_state_dict(self, state_dict):
      """Load pretreained params. Ignore """
      own_state = self.state_dict()

      for name, param in state_dict.items():
          # If the param is unknown or it belongs to the original classifier layer of resnet.
          # Note: we now have 2 classifier layer with different names (class_classifier and domain_classifier)
          #if name not in own_state:
          try:
              #print(name)
              #if name.startswith("conv") or name.startswith("bn") or name.startswith("relu") or name.startswith("maxpool") or name.startswith("layer"):
              # IF here we are processing the "classifier" parameters of alexnet.
              # Copy the parameter to both domain_classifier and label_classifier
              # Just if the saved params matches dimension of current layer state (i.e. FC layer often (like in this excercise) are of different size)
              ec_name, ed_name = self._rename_layer(name)  # Adapt to new named resnet layer (ec and ed prefix)
              if len(param) == len(own_state.get(ec_name, [])):
                own_state[ec_name].copy_(param)
                #print("YEES:", name, ec_name)
              if len(param) == len(own_state.get(ed_name, [])):
                own_state[ed_name].copy_(param)
                #print("YEES:", name, ed_name)
              else:
                #print("NOOO", name, ec_name, ed_name)
                print(len(param), len(own_state.get(ed_name, [])))
            # In every case go to the next iteration. In fact or the param was ignored
            # Or the weight/bias was already loaded (copied)
          except Exception as e:
            print(e)
            continue
          if isinstance(param, Parameter):
              # print("Old param", name)
              # backwards compatibility for serialized parameters
              param = param.data
              if len(param) == len(own_state.get(ec_name, [])):
                own_state[ec_name].copy_(param)
                #print("Loaded old param_c")
              if len(param) == len(own_state.get(ed_name, [])):
                own_state[ed_name].copy_(param)
                #print("Loaded old param_d")

    def heads_xavier_init(self):
      for layer in self.pretexthead:
        try:
          torch.nn.init.xavier_uniform(layer.weight)
        except Exception:
          print("[INFO] Xavier initialization cant be applied to the layer: ", layer)
          continue

      for layer in self.mainhead:
        try:
          torch.nn.init.xavier_uniform(layer.weight)
        except Exception:
          print("[INFO] Xavier initialization cant be applied to the layer: ", layer)
          continue

'''
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)
'''


'\ndef _resnet(arch, block, layers, pretrained, progress, **kwargs):\n    model = ResNet(block, layers, **kwargs)\n    if pretrained:\n        state_dict = load_state_dict_from_url(model_urls[arch],\n                                              progress=progress)\n        model.load_state_dict(state_dict)\n    return model\n\n\ndef resnet18(pretrained=False, progress=True, **kwargs):\n    r"""ResNet-18 model from\n    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_\n\n    Args:\n        pretrained (bool): If True, returns a model pre-trained on ImageNet\n        progress (bool): If True, displays a progress bar of the download to stderr\n    """\n    return _resnet(\'resnet18\', BasicBlock, [2, 2, 2, 2], pretrained, progress,\n                   **kwargs)\n'

# Training

## Load model and parameters, define optimizer and scheduler

In [ ]:
class EntropyLoss(nn.Module):
    
    def __init__(self):
        super(EntropyLoss,self).__init__()
        
    def forward(self, input, target):
        return self._entropy_loss(input)

    def _entropy_loss(self, logits):
        p_softmax = F.softmax(logits, dim=1)
        mask = p_softmax.ge(0.000001)  # greater or equal to, used for numerical stability
        mask_out = torch.masked_select(p_softmax, mask)
        entropy = -(torch.sum(mask_out * torch.log(mask_out)))
        return entropy / float(p_softmax.size(0))

In [ ]:
# Load state dict
state_dict = load_state_dict_from_url(model_urls['resnet18'])

# Load model
net = RGBD_DA(BasicBlock, [2, 2, 2, 2] )
net.custom_load_state_dict(state_dict)
net.heads_xavier_init()
net.mainhead[4].register_forward_hook(lambda m, inp, out: F.dropout(out, p=DROP_OUT, training=m.training))
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
criterion_DA = EntropyLoss()  # We dont use label for target => we use entropy

# Choose parameters to optimize
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of the model

# Define optimizer: An optimizer updates the weights based on loss
# SGD with momentum (or eventually with nesterov momentum)
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY, nesterov=NESTEROV)
#optimizer = optim.Adam(parameters_to_optimize, lr=LR, betas=BETAS, eps=EPS, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate (common one is step-down that updates each step LR = oldLR*Gamma)
scheduler = None #optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)



The size of tensor a (512) must match the size of tensor b (64) at non-singleton dimension 1
too many values to unpack (expected 2)
too many values to unpack (expected 2)
[INFO] Xavier initialization cant be applied to the layer:  BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[INFO] Xavier initialization cant be applied to the layer:  ReLU()
[INFO] Xavier initialization cant be applied to the layer:  BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[INFO] Xavier initialization cant be applied to the layer:  ReLU()
[INFO] Xavier initialization cant be applied to the layer:  View()
[INFO] Xavier initialization cant be applied to the layer:  BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
[INFO] Xavier initialization cant be applied to the layer:  ReLU()
[INFO] Xavier initialization cant be applied to the layer:  AdaptiveAvgPool2d(output_size=(1, 1))
[INFO] Xavier initialization cant be app

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:391: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:398: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


## Define training procedure

In [ ]:
def run_inference(net, step, tensorboard_writer=None, verbose=1):
  # **** At the end of each epoch perform evaluation
    net.train(False)
    running_corrects1 = 0
    running_corrects2 = 0

    eval_step = 1
    with torch.no_grad(): 
      for rgb_images, depth_images, trgb_images, tdepth_images, labels, relative_rotations in tqdm(target_test_dataloader):
        rgb_images = rgb_images.to(DEVICE)
        labels = labels.to(DEVICE)
        trgb_images, tdepth_images = trgb_images.to(DEVICE), tdepth_images.to(DEVICE)

        depth_images = depth_images.to(DEVICE)
        relative_rotations = relative_rotations.to(DEVICE)

        _, discriminator_outputs = net(trgb_images, tdepth_images, forward_to_M=False, forward_to_P=True)
        class_outputs, _ = net(rgb_images, depth_images, forward_to_M=True, forward_to_P=False)

        _, preds1 = torch.max(class_outputs.data, 1)  # Get predictions
        _, preds2 = torch.max(discriminator_outputs.data, 1)  # Get predictions

        # Update Corrects
        running_corrects1 += torch.sum(preds1 == labels.data).data.item()
        running_corrects2 += torch.sum(preds2 == relative_rotations.data).data.item()

        eval_step +=1

      # Calculate Accuracy
      accuracy1 = running_corrects1 / float(len(target_test_datasubset))
      accuracy2 = running_corrects2 / float(len(target_test_datasubset))

      
      if verbose:
        print('[INFO] Validation objclass Accuracy (@step:{}): {}'.format(step, accuracy1))
        print('[INFO] Validation rotation Accuracy (@step:{}): {}'.format(step, accuracy2))
        now = datetime.now()
        print('[INFO] Current time (@step:{}): {}'.format(step, now))

      # /--- Log to tensorboard
      if tensorboard_writer:
          tensorboard_writer.add_scalar('Val. Accuracy (objclass)', accuracy1, (step) )
          tensorboard_writer.add_scalar('Val. Accuracy (rotation)', accuracy2, (step) )

          tensorboard_writer.flush()  # Write to disk and update tensorboard. could use flush_secs as writer init argument
      # ---/ Log to tensorboard
      return accuracy1, accuracy2

In [ ]:
#@title Training Implementation
import gc
from numpy.random import default_rng
from datetime import datetime


def train_and_get_best(net, num_epochs, scheduler, verbose=False, tensorboard_writer=None, cudnn_enabled=True, source_only=False, only_mainhead=False, gradient_accumulation_step=1):
  """ Train the network with the specified parameters and data.

  @return:
    (bestnet, bestaccuracy, accuracy_for_epoch, )

  TODO: when cudnn will be fixed avoid cudnn.enabled=False for improved performance 
      (this limitation is applied only when tensorboard is enabled)
  """
  # Initialize the random generator useful to get some random sample for evaluation plot (tensorboard) after each epoch
  #rng = default_rng()
  #random_img_batch_eval_idx = int(rng.random(1)*len(target_val_dataloader)+1)

  net = net.to(DEVICE)

  cudnn.benchmark = cudnn_enabled # set to fase to increase time but reduce memory usage

  # Keep best accuracy to check condition for updating best_net
  best_accuracy = 0
  current_step = 0
  accuracy_for_epoch = {}

  # Start iterating over the epochs
  for epoch in range(num_epochs):
    if verbose and scheduler is not None:
      print('Starting epoch {}/{}, LR = {}'.format(epoch+1, num_epochs, scheduler.get_lr()))

    # Iterate over the dataset (zip?)
    for rgb_images, depth_images, t_rgb_images, t_depth_images, labels, relative_rotations in source_train_dataloader:

      rgb_images,labels = rgb_images.to(DEVICE), labels.to(DEVICE)
      depth_images, relative_rotations = depth_images.to(DEVICE), relative_rotations.to(DEVICE)
      t_rgb_images, t_depth_images = t_rgb_images.to(DEVICE), t_depth_images.to(DEVICE)
      net.train() 
      
      optimizer.zero_grad() # Zero-ing the gradients

      # NOTE: USING THE SAME Loss criterion for all 
      # Stage A) Train with source dataset (labeled and NOT transformed)
      
      class_outputs, _ = net(rgb_images, depth_images, forward_to_M=True, forward_to_P=False) # Forward pass to the network to M head
      loss_m = criterion(class_outputs, labels)  # Compute loss for M based on output and ground truth obj label
      loss_m.backward()
      if not only_mainhead: 
        _ , discriminator_outputs = net(t_rgb_images, t_depth_images, forward_to_M=False, forward_to_P=True)
        loss_p_source = criterion_DA(discriminator_outputs, _) #first component in loss_p
        #loss_p_source = criterion(class_outputs, labels) #first component in loss_p

        if current_step % LOG_FREQUENCY == 0 and verbose == True:
          print('Step {}, Loss M on Source {}'.format(current_step, loss_m.item()))
          print('Step {}, Loss P on Source {}'.format(current_step, loss_p_source.item()))

        if tensorboard_writer:
            tensorboard_writer.add_scalar('Train Loss1', loss_m.item(), (current_step) )
            tensorboard_writer.add_scalar('Train Loss2', loss_p_source.item(), (current_step) )

        if not source_only: # it does not refer to the baseline, but to the dataset source
          # Stage C) Train with target dataset
          rgb_images, depth_images, t_rgb_images, t_depth_images, labels, relative_rotations = next(iter(target_train_dataloader))
          
          rgb_images,labels = rgb_images.to(DEVICE), labels.to(DEVICE)
          depth_images, relative_rotations = depth_images.to(DEVICE), relative_rotations.to(DEVICE)
          t_rgb_images, t_depth_images = t_rgb_images.to(DEVICE), t_depth_images.to(DEVICE)

          _ , discriminator_outputs = net(t_rgb_images, t_depth_images, forward_to_M=False, forward_to_P=True)
          
          loss_p_target = criterion_DA(discriminator_outputs, _)
          if current_step % LOG_FREQUENCY == 0 and verbose == True:
            print('Step {}, Loss Pretext on Target {}'.format(current_step, loss_p_target.item()))
          
          if tensorboard_writer:
            tensorboard_writer.add_scalar('Train Loss Pretext Target', loss_p_target.item(), (current_step+1) )
          
          loss_p = (loss_p_source + loss_p_target) * LAMBDA
        else:
          loss_p = (loss_p_source) * LAMBDA
        
        loss_p.backward()

      optimizer.step() # update weights based on accumulated gradients

      # Eventually compute running loss to log on tensorboard
      # running_loss += loss.item()
      current_step += 1
      if current_step % VAL_FREQUENCY == 0:
        run_inference(net, current_step, tensorboard_writer=tensorboard_writer, verbose=verbose)
    
    accuracy1, accuracy2 = run_inference(net, current_step, tensorboard_writer=tensorboard_writer, verbose=verbose)
    accuracy_for_epoch[epoch+1] = accuracy1
    net.train(True)  # Restore train flag

    # Save the model with the best accuracy to use it against the testset
    if accuracy1>best_accuracy:
        best_net, best_accuracy = copy.deepcopy(net), accuracy1
        if verbose:
          print('[INFO] New best accuracy: ', accuracy1, " (@epoch ",(epoch+1), ") !")
    # Step the scheduler
    if scheduler is not None:
        scheduler.step() 

    # Force to run garbace collector to help avoid oom
    gc.collect()
  
  last_accuracy = accuracy1
  return (last_accuracy, best_net, best_accuracy, accuracy_for_epoch)

## Start training


In [ ]:
net = net.to(DEVICE) 
writer = SummaryWriter('runs/RGB_DA_RandomCropTrain_min_size=42_ds_size_417k18k_dropout_nostepdown')  # `log_dir` set to "runs" 
#writer = None  # Uncomment to DISABLE tensorboard logging

# Start training
accuracy_model, best_net, best_accuracy, accuracy_for_epoch = \
train_and_get_best(net, NUM_EPOCHS, scheduler, verbose=True,cudnn_enabled=True, tensorboard_writer=writer)


Step 0, Loss M on Source 4.710307598114014
Step 0, Loss P on Source 1.137976884841919
Step 0, Loss Pretext on Target 1.1442320346832275
Step 10, Loss M on Source 4.818904399871826
Step 10, Loss P on Source 1.1825305223464966
Step 10, Loss Pretext on Target 1.1377079486846924
Step 20, Loss M on Source 4.670499801635742
Step 20, Loss P on Source 1.1667165756225586
Step 20, Loss Pretext on Target 1.1827263832092285
Step 30, Loss M on Source 4.651278972625732
Step 30, Loss P on Source 1.202904462814331
Step 30, Loss Pretext on Target 1.1710126399993896
Step 40, Loss M on Source 4.248286247253418
Step 40, Loss P on Source 1.120800256729126
Step 40, Loss Pretext on Target 1.1439311504364014
Step 50, Loss M on Source 3.8982040882110596
Step 50, Loss P on Source 1.1216094493865967
Step 50, Loss Pretext on Target 1.1182715892791748
Step 60, Loss M on Source 3.9110429286956787
Step 60, Loss P on Source 1.2018189430236816
Step 60, Loss Pretext on Target 1.1738883256912231
Step 70, Loss M on Sourc

100%|██████████| 32/32 [00:06<00:00,  4.70it/s]


[INFO] Validation objclass Accuracy (@step:150): 0.21559191530317612
[INFO] Validation rotation Accuracy (@step:150): 0.25601539942252166
[INFO] Current time (@step:150): 2020-06-26 10:39:49.886240
Step 150, Loss M on Source 3.1459004878997803
Step 150, Loss P on Source 1.1684775352478027
Step 150, Loss Pretext on Target 0.9759946465492249
Step 160, Loss M on Source 3.0577313899993896
Step 160, Loss P on Source 1.1760430335998535
Step 160, Loss Pretext on Target 0.9208938479423523
Step 170, Loss M on Source 3.2682130336761475
Step 170, Loss P on Source 1.1519107818603516
Step 170, Loss Pretext on Target 0.8758268356323242
Step 180, Loss M on Source 2.993586301803589
Step 180, Loss P on Source 1.1333630084991455
Step 180, Loss Pretext on Target 0.8888252973556519
Step 190, Loss M on Source 3.1995797157287598
Step 190, Loss P on Source 1.1917312145233154
Step 190, Loss Pretext on Target 0.8270378112792969
Step 200, Loss M on Source 2.6844654083251953
Step 200, Loss P on Source 1.17670810

100%|██████████| 32/32 [00:06<00:00,  4.66it/s]


[INFO] Validation objclass Accuracy (@step:300): 0.3753609239653513
[INFO] Validation rotation Accuracy (@step:300): 0.24157844080846969
[INFO] Current time (@step:300): 2020-06-26 10:47:14.784965
Step 300, Loss M on Source 2.541025400161743
Step 300, Loss P on Source 1.1415863037109375
Step 300, Loss Pretext on Target 0.48476648330688477
Step 310, Loss M on Source 2.131532669067383
Step 310, Loss P on Source 1.1170985698699951
Step 310, Loss Pretext on Target 0.4822668731212616
Step 320, Loss M on Source 2.529156446456909
Step 320, Loss P on Source 1.1711063385009766
Step 320, Loss Pretext on Target 0.5012309551239014
Step 330, Loss M on Source 2.188433885574341
Step 330, Loss P on Source 1.1416854858398438
Step 330, Loss Pretext on Target 0.4333248436450958
Step 340, Loss M on Source 1.8831267356872559
Step 340, Loss P on Source 1.1151281595230103
Step 340, Loss Pretext on Target 0.4149876534938812
Step 350, Loss M on Source 2.3429434299468994
Step 350, Loss P on Source 1.10934615135

...

100%|██████████| 32/32 [00:06<00:00,  4.61it/s]


[INFO] Validation objclass Accuracy (@step:11550): 0.4542829643888354
[INFO] Validation rotation Accuracy (@step:11550): 0.2569778633301251
[INFO] Current time (@step:11550): 2020-06-26 20:04:30.054482
Step 11550, Loss M on Source 0.765507698059082
Step 11550, Loss P on Source 0.7670464515686035
Step 11550, Loss Pretext on Target 0.7296093106269836
Step 11560, Loss M on Source 0.81977778673172
Step 11560, Loss P on Source 0.7978391051292419
Step 11560, Loss Pretext on Target 0.7291103005409241
Step 11570, Loss M on Source 0.7331459522247314
Step 11570, Loss P on Source 0.78643798828125
Step 11570, Loss Pretext on Target 0.7306705713272095
Step 11580, Loss M on Source 0.6412111520767212
Step 11580, Loss P on Source 0.7901175022125244
Step 11580, Loss Pretext on Target 0.7297459244728088
Step 11590, Loss M on Source 0.9522562026977539
Step 11590, Loss P on Source 0.7688664197921753
Step 11590, Loss Pretext on Target 0.7297846674919128
Step 11600, Loss M on Source 0.8051034808158875
Step 

100%|██████████| 32/32 [00:06<00:00,  4.66it/s]


[INFO] Validation objclass Accuracy (@step:11700): 0.45139557266602504
[INFO] Validation rotation Accuracy (@step:11700): 0.2685274302213667
[INFO] Current time (@step:11700): 2020-06-26 20:12:05.586977
Step 11700, Loss M on Source 0.6665953397750854
Step 11700, Loss P on Source 0.8117145299911499
Step 11700, Loss Pretext on Target 0.7349039316177368
Step 11710, Loss M on Source 0.7771956920623779
Step 11710, Loss P on Source 0.7753667831420898
Step 11710, Loss Pretext on Target 0.7365744709968567
Step 11720, Loss M on Source 0.8326146006584167
Step 11720, Loss P on Source 0.7776907682418823
Step 11720, Loss Pretext on Target 0.7367501258850098
Step 11730, Loss M on Source 0.637851893901825
Step 11730, Loss P on Source 0.8072383403778076
Step 11730, Loss Pretext on Target 0.7394046187400818
Step 11740, Loss M on Source 0.8782002329826355
Step 11740, Loss P on Source 0.804417073726654
Step 11740, Loss Pretext on Target 0.7381858825683594


KeyboardInterrupt: ignored

In [ ]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
print("now =", now)

In [ ]:
ch.tensorboard_backup()

In [ ]:
ch.torch_model_state_backup(net, "RGB_DA_RandomCropTrain_min_size=42_ds_size_417k18k_dropout_nostepdown")